# Example serial code for Component 2

This notebook demonstrates Monte Carlo Simulation

## Why Monte Carlo (MC) simulation?

We use MC simulations for problems where we want to find numerical values, where the process is subject to computational costs.

This is why MCMC (Markov chain Monte Carlo) is used for Bayesian regression -- each parameter space has an exponentially large search space that would be intractable to compute on every possible hypothesis.

Monte Carlo simulation: Repeated random sampling to obtain numerical results.

## Basic steps of a Monte Carlo simulation:

1. Define the model and inputs
2. Randomly generate the inputs using a stochastic process
3. Using the inputs in a deterministic process to produce outputs
4. Aggregating and analyzing the results

### Using MC to numerically obtain the value of $\pi$ with a unit circle

We can numerically obtain the value of $\pi$ by running a MC simulation with the following model:
1. Assuming the [unit circle](https://en.wikipedia.org/wiki/Unit_circle), then we know that everywhere along the unit circle, the hypotenuse is 1: $x^2 + y^2 = 1$.  We also know that the area of a unit circle should be $\pi$, since $a_{circle} = \pi \cdot r^2$.
2. Since our model is simply just $z = x^2 + y^2$, where the constraint for accepted values is $z \le 1$, choose distributions and bounds for the parameters.
3. We assume that the ratio between accepted values and total ($c_{accepted} / c_{total}$) attempted will be $\pi / 4$, since we are computing the area of the circle as a ratio to the area of the square.  Thus, $\pi = 4 \cdot c_{accepted} / c_{total}$.
4. Draw samples until the delta of the computed value for $\pi$ between samples is less than some accepted numerical error.

Let's write this up as Python code using `scipy.stats`!  Keep track of the value of the computed $\pi$ between samples so we can plot it!

In [ ]:
import numpy as np
import scipy.stats
from typing import Union

In [ ]:
def circle_accept(x: Union[float, np.ndarray],
                  y: Union[float, np.ndarray]) -> Union[bool, np.ndarray]:
    """
    circle_accept(x, y) will return whether the dart thrown at (x, y)
      is within the area of the unit circle.
    """
    return np.square(x) + np.square(y) <= 1

In [ ]:
def monte_carlo_pi(num_draws: int = 1000000) -> float:
    x_dist = scipy.stats.uniform(-1, 2)  # Uniform[-1, 1]
    y_dist = scipy.stats.uniform(-1, 2)  # Uniform[-1, 1]

    the_rng = np.random.default_rng()
    x_dist.random_state = the_rng
    y_dist.random_state = the_rng

    x_draw = x_dist.rvs(size=num_draws)
    y_draw = y_dist.rvs(size=num_draws)

    z_accept = circle_accept(x_draw, y_draw)
    count_accepted = z_accept.sum()

    pi_estimate = 4 * (count_accepted / num_draws)
    return pi_estimate

In [ ]:
# %%timeit -n10

monte_carlo_pi(100000000)


As we experiment with the number of draws (`num_draws`), we can get closer and closer to the "real" value of $\pi$.

Now it's your turn!  Based on the code written here -- what can we parallelize?  What are "independent" computations in this particular problem?

Write your code in the following cell (this assumes Python, but feel free to change as needed):

In [ ]:
%%writefile ParallelCode.py

print("Replace this with your Python code!!")

def main() -> None:
    print("Replace this with calls to your functions")

if __name__ == '__main__':
    print("Replace this with proper timeit calls")

Use the `%%qsub` Jupyter cell command from `cfxmagic` to submit jobs below.  If you are not using Python, replace the line with the correct call.

In [ ]:
import cfxmagic

In [ ]:
%%qsub 
cd $PBS_O_WORKDIR
python ParallelCode.py

Now we can check `qstat` to see the status of our job.  Feel free to run the following cell repeatedly until our job disappears from the list.

In [ ]:
!qstat

We can now retrieve our output and error streams if they exist, and they will be named STDIN due to the `%%qsub` magic:

In [ ]:
%cat STDIN.o*

In [ ]:
%cat STDIN.e*

Once you have verified that your parallel code works properly, you can continue to the third component, which covers analyzing your code.